In [1]:
import os
import sys
import glob
import dlib
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np

In [ ]:
# 利用标注好的xml 训练svm分类器
faces_folder = "./handpredictor"
#“handdetector.svm”为已训练好的，如果存在择不用再训练
if not os.path.exists("handdetector.svm"):
    options = dlib.simple_object_detector_training_options()#//设置训练参数
    options.add_left_right_image_flips = True
    options.C = 5
    options.num_threads = 4
    options.be_verbose = True

    training_xml_path = os.path.join(faces_folder, "handsdata.xml")#训练数据集
    testing_xml_path = os.path.join(faces_folder, "test.xml")#测试数据集
    dlib.train_simple_object_detector(training_xml_path, "handdetector.svm", options)#生成检测器“handdetector.svm”

    print("")  # Print blank line to create gap from previous output
    print("Training accuracy: {}".format(
    dlib.test_simple_object_detector(training_xml_path, "handdetector.svm")))
    print("Testing accuracy: {}".format(
    dlib.test_simple_object_detector(testing_xml_path, "handdetector.svm")))



In [ ]:
# 目标检测 测试
detector = dlib.simple_object_detector("handdetector.svm")#加载检测器
'''
# We can look at the HOG filter we learned.  It should look like a face.  Neat!
win_det = dlib.image_window()
win_det.set_image(detector)
'''
# Now let's run the detector over the images in the faces folder and display the
# results.
print("Showing detections on the images in the faces folder...")
win = dlib.image_window()
#测试“testimages”目录下的所有jpg文件
for f in glob.glob(os.path.join(faces_folder, "test/*.jpg")):
    print("Processing file: {}".format(f))
    img = dlib.load_rgb_image(f)
    dets = detector(img)
    print("Number of faces detected: {}".format(len(dets)))
    for k, d in enumerate(dets):
        print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
            k, d.left(), d.top(), d.right(), d.bottom()))

    win.clear_overlay()
    win.set_image(img)
    win.add_overlay(dets)
    dlib.hit_enter_to_continue()

In [ ]:
faces_folder = "./handpredictor"
#“handpredictor.dat”为已训练好的，如果存在择不用再训练
if not os.path.exists("handpredictor.dat"):
    options = dlib.shape_predictor_training_options()
    #现在让对象负责训练模型。
    #这个算法有很多参数，你可以搞砸。这个documentation for the shape_predictor_trainer的文档解释了所有这些。
    #您还应该阅读Kazemi的论文，其中解释了所有参数非常详细。但是，这里我只设置其中的三个与默认值不同
    #我这么做是因为我们有一个非常小的数据集。特别地，设置过采样
    #到较高的数量（300）有效地提高了训练集的大小，所以
    #这有助于这个示例。

    #通过对训练样本进行随机变形扩大样本数目。比如你原来有N张训练图片，通过该参数的设置，训练样本数将变成N*_oversampling_amount张。
    #所以通常该值越大越好，只是训练耗时也会越久
    options.oversampling_amount = 100
    #我还通过显式增加来减少模型的容量
    #正则化（使nu更小）以及使用较小的深度。
    #正则项，nu越大，表示对训练样本fit越好，当然也越有可能发生过拟合。_nu取值范围(0,1]，默认取0.1。
    options.nu = 0.05
    #树深，则树的叶子节点个数为2(_tree_depth)个
    options.tree_depth = 4
    #cpu测试机器为num_threads(线程)核数
    options.num_threads=8
    options.be_verbose = True

    #dlib.train_shape_predictor() 执行实际的训练。输入是一个XML文件，它列出了训练数据集中的图像
    #并且还包含人脸的位置部分零件。
    training_xml_path = os.path.join(faces_folder, "handsdata.xml")
    #它将保存final predictor to handpredictor.dat.
    dlib.train_shape_predictor(training_xml_path, "handpredictor.dat", options)

# 现在我们有了一个模型，可以测试它了 dlib.test_shape_predictor()
# measures the average distance between a face landmark output by the
# shape_predictor and where it should be according to the truth data.
    print("\nTraining accuracy: {}".format(
        dlib.test_shape_predictor(training_xml_path, "handpredictor.dat")))
        # 真正的测试是看它在没有经过训练的数据上表现得有多好。
        #我们在一个非常小的数据集上训练它，所以精度不是很高。
        #但是它仍然做得很好. 此外，如果你在一个大型飞机上训练
        # 面部标志数据集，您将获得最先进的结果, 如Kazemi论文所示.
    testing_xml_path = os.path.join(faces_folder, "test.xml")
    print("Testing accuracy: {}".format(
        dlib.test_shape_predictor(testing_xml_path, "handpredictor.dat")))


In [3]:
def cv2ImgAddText(img,text,left, top, textColor=(0, 255, 0), textSize=20):
    if (isinstance(img, np.ndarray)):  #判断是否OpenCV图片类型
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img)
    fontText = ImageFont.truetype("font/simsun.ttc", textSize, encoding="utf-8")
    draw.text((left, top), text, textColor, font=fontText)
    return cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2BGR)


# 现在让我们像在普通应用程序中一样使用它.  首先，我们将从磁盘加载它。
# 我们还需要加载一个面部检测器来提供初始值面部位置的估计。
predictor = dlib.shape_predictor("handpredictor.dat")
#detector = dlib.get_frontal_face_detector()
#加载之前训练好的目标检测器
detector = dlib.simple_object_detector("handdetector.svm")############################


#现在让我们在面部的图像上运行检测器和shape_predictor文件夹并显示结果。
#print("Showing detections and predictions on the images in the faces folder...")
win = dlib.image_window()
for f in glob.glob(os.path.join(faces_folder, "test/*.jpg")):
    #print("Processing file: {}".format(f))
    img = dlib.load_rgb_image(f)
    #img = dlib.load_rgb_image(os.path.join(faces_folder, "handimages/S1-P1-F-11-1.jpg"))

    # 让探测器找出每个人脸的边框。 The 1 in the
    # 第二个参数指示我们应该对图像进行1次上采样. 
    #这将使所有东西变得更大，并允许我们检测更多的脸。
    #dets = detector(img, 1)
    dets = detector(img)
    #d=dlib.rectangle(114, 140, 355, 388)
    #print("Left: {} Top: {} Right: {} Bottom: {}".format(d.left(), d.top(), d.right(), d.bottom()))
    # Get the landmarks/parts for the face in box d.
    for d in dets:
        shape = predictor(img, d)
        cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        #特征点总数
        partss=len(shape.parts())
        Li=["十宣穴1","十宣穴2","十宣穴3","十宣穴4","十宣穴5","少府穴","劳宫穴","鱼际穴"]        
        for i in range(8):
            name = Li[i]
            pa=shape.part(i)
            p = (pa.x,pa.y)
#             #(shape.part(i).x, shape.part(i).y)
#             cv2.circle(img,p,2,(0,255,0),2)
#             #各参数依次是：照片/添加的文字/左上角坐标/字体/字体大小/颜色/字体粗细
#             #cv2.putText(img,name,p,cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),1)
#             frame =cv2ImgAddText(img,name,p,cv2.FONT_HERSHEY_SIMPLEX,(255,255,0), 12)
            
            cv2.circle(img, p, 5, color=(0,255, 0))
            # 利用cv2.putText输出1-68
            font = cv2.FONT_HERSHEY_SIMPLEX
            img =cv2ImgAddText(img,name,pa.x,pa.y,(0,0,0),40)
                    
                    
    # Draw the face landmarks on the screen.
    cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    win.clear_overlay()
    win.set_image(img)
    #win.add_overlay(shape)
    win.add_overlay(d)
    dlib.hit_enter_to_continue()